In [2]:
import numpy as np
from keras.engine import Input
from keras.layers import merge, Embedding, Dropout, Conv1D, Lambda, LSTM, Dense, concatenate, TimeDistributed
from keras import backend as K
from keras.models import Model
import tensorflow as tf
import pandas as pd
from time import strftime, gmtime, time
from keras.layers import Conv2D
import random

In [18]:
# loads the embeddings variable from the tensorflow checkpoint
def load_embeddings(checkpoint_path):
    with tf.Session() as session:
        if checkpoint_path:
            saver = tf.train.import_meta_graph(checkpoint_path+'/linear_D1024.ckpt-4.meta')
            saver.restore(session, tf.train.latest_checkpoint(checkpoint_path))
          
            return session.run('embedding:0')

In [23]:
embeddings_trained=load_embeddings('../chord2vec/save_models/linear')

INFO:tensorflow:Restoring parameters from ../chord2vec/save_models/linear\linear_D1024.ckpt-4


In [24]:
type(embeddings_trained)

numpy.ndarray

In [3]:
question = Input(shape=(149,), dtype='int32', name='question_base')
answer_good = Input(shape=(149,), dtype='int32', name='answer_good_base')


In [4]:
import pickle

with open('./QA_data/Chord_embedding_sum_lookup0503', 'rb') as pickle_load:
    embeddings_trained = pickle.load(pickle_load)
print('ok')

ok


In [7]:
# add embedding layers

embedding = Embedding(input_dim=22,
                      output_dim=embeddings_trained.shape[1],
                      weights=[embeddings_trained])
question_embedding = embedding(question)
answer_embedding = embedding(answer_good)


In [8]:

hidden_layer = TimeDistributed(Dense(200, activation='tanh'))

question_hl = hidden_layer(question_embedding)
answer_hl = hidden_layer(answer_embedding)

# cnn
cnns = [Conv1D(kernel_size=kernel_size,
                       filters=1000,
                       activation='tanh',
                       padding='same') for kernel_size in [2, 3, 5, 7]]
# question_cnn = merge([cnn(question_embedding) for cnn in cnns], mode='concat')
question_cnn = concatenate([cnn(question_hl) for cnn in cnns], axis=-1)
# answer_cnn = merge([cnn(answer_embedding) for cnn in cnns], mode='concat')
answer_cnn = concatenate([cnn(answer_hl) for cnn in cnns], axis=-1)

# maxpooling
maxpool = Lambda(lambda x: K.max(x, axis=1, keepdims=False), output_shape=lambda x: (x[0], x[2]))
maxpool.supports_masking = True
# enc = Dense(100, activation='tanh')
# question_pool = enc(maxpool(question_cnn))
# answer_pool = enc(maxpool(answer_cnn))
question_pool = maxpool(question_cnn)
answer_pool = maxpool(answer_cnn)

In [12]:
model = Model(inputs=[question, answer_good], outputs=[question_pool,answer_pool])

In [13]:
def save_epoch(epoch,model):
    if not os.path.exists('models/'):
        os.makedirs('models/')
    model.save_weights('models/weights_epoch_%d.h5' % epoch, overwrite=True)
def get_time():
    return strftime('%Y-%m-%d %H:%M:%S', gmtime())

In [14]:
val_loss = {'loss': 1., 'epoch': 0}


In [15]:
import pickle

with open('./QA_data/questions0501', 'rb') as pickle_load:
    Q = pickle.load(pickle_load)
with open('./QA_data/good_answers0501', 'rb') as pickle_load:
    good_Ans = pickle.load(pickle_load)
with open('./QA_data/all_answers0501', 'rb') as pickle_load:
    all_answers = pickle.load(pickle_load)
print('ok')

ok


In [18]:
model.compile(optimizer='adam')


ValueError: The model cannot be compiled because it has no loss to optimize.

In [17]:
nb_epoch=100
for i in range(1, nb_epoch+1):
    # sample from all answers to get bad answers
    # if i % 2 == 0:
    #     bad_answers = self.pada(random.sample(self.answers.values(), len(good_answers)))
    # else:
    #     bad_answers = self.pada(get_bad_samples(indices, top_50))

    #random.sample第一個參數吃list
    #self.answers.values():dist
    print('Sample bad_answers...')
    #bad_answers = random.sample(all_answers, len(good_Ans))

    print('Fitting epoch %d' % i)
    hist = model.fit([Q, good_Ans], epochs=1, batch_size=100,validation_split=0.1, verbose=1)

    if hist.history['val_loss'][0] < val_loss['loss']:
        val_loss = {'loss': hist.history['val_loss'][0], 'epoch': i}
    print('%s -- Epoch %d ' % (get_time(), i) +'Loss = %.4f, Validation Loss = %.4f ' % (hist.history['loss'][0], hist.history['val_loss'][0]) +'(Best: Loss = %.4f, Epoch = %d)' % (val_loss['loss'], val_loss['epoch']))

    save_epoch(i,model)
best_loss=val_loss

Sample bad_answers...
Fitting epoch 1


RuntimeError: You must compile your model before using it.

In [33]:
len(Q)

6793